<a href="https://colab.research.google.com/github/dataplusplus-ai/Computer_Vision_Deep_Learning/blob/main/densenet_vgg16_19ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/trainjpg"
!unzip -q "/content/gdrive/My Drive/testjpg"
!unzip -q "/content/gdrive/My Drive/validationjpg"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)

0.25

In [ ]:
X_smote, y_smote = sm.fit_resample(X_train, y_train)

94.0

In [ ]:
%tensorflow_version 1.x

import numpy as np
import pandas as pd
import cv2
import json

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from __future__ import print_function
import keras
from keras import models
from keras import layers
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import MaxPool2D , Flatten
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model

import os
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import SGD
from keras.utils import plot_model

Using TensorFlow backend.


In [ ]:
# 25 % of healthy_wheat 142 images are 36 so val healthy_wheat 36
# 25 % of leaf_rust 357 images are 90
# 25 % of stem_rust 376 images are 94

In [ ]:
from PIL import Image
import os

In [ ]:
#print('no of test images is {}'.format(len(os.listdir('/content/testjpg'))))
print('no of train images is {}+{}+{}'.format(len(os.listdir('/content/trainjpg/healthy_wheat')),
                                               len(os.listdir('/content/trainjpg/stem_rust')),
                                              len(os.listdir('/content/trainjpg/leaf_rust'))))
print('no of validation images is {}+{}+{}'.format(len(os.listdir('/content/validationjpg/healthy_wheat')),
                                               len(os.listdir('/content/validationjpg/stem_rust')),
                                              len(os.listdir('/content/validationjpg/leaf_rust'))))

no of train images is 106+282+267
no of validation images is 36+94+90


In [ ]:
# from imgaug import augmenters as iaa

# aug1 = iaa.CoarseDropout(p=0.10, size_percent=0.05)

# def additional_augmenation(image):
#     image = aug1.augment_image(image)

#     return image

In [ ]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):

  """
  
  p : the probability that random erasing is performed
  s_l, s_h : minimum / maximum proportion of erased area against input image
  r_1, r_2 : minimum / maximum aspect ratio of erased area
  v_l, v_h : minimum / maximum value for erased area
  pixel_level : pixel-level randomization for erased area
  """

  def eraser(input_img):
    img_h, img_w, img_c = input_img.shape
    p_1 = np.random.rand()

    if p_1 > p:
        return input_img

    while True:
        s = np.random.uniform(s_l, s_h) * img_h * img_w
        r = np.random.uniform(r_1, r_2)
        w = int(np.sqrt(s / r))
        h = int(np.sqrt(s * r))
        left = np.random.randint(0, img_w)
        top = np.random.randint(0, img_h)

        if left + w <= img_w and top + h <= img_h:
            break

    if pixel_level:
        c = np.random.uniform(v_l, v_h, (h, w, img_c))
    else:
        c = np.random.uniform(v_l, v_h)

    input_img[top:top + h, left:left + w, :] = c

    return input_img

  return eraser

In [ ]:
img_width, img_height = 224,224
batch_size=32
train_datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=2,
        # randomly shift images horizontally
        width_shift_range=0.08,
        # randomly shift images vertically
        height_shift_range=0.05,
        # set range for random shear
        shear_range=0.07,
        # set range for random zoom
        zoom_range=0.05,
        # set range for random channel shifts
        channel_shift_range=0.07,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        rescale=1. / 255,
        # set rescaling factor (applied before any other transformation)
        # set function that will be applied on each input
        #Cutout Implementation
        preprocessing_function=get_random_eraser(p=0.08,v_l=0, v_h=0.3, s_l=0.008, s_h=0.05, pixel_level=False),
        #preprocessing_function=additional_augmenation, 
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

In [ ]:
img_width, img_height = 224,224
batch_size=32
validation_datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=30,
        # randomly shift images horizontally
        width_shift_range=0.08,
        # randomly shift images vertically
        height_shift_range=0.05,
        # set range for random shear
        shear_range=0.07,
        # set range for random zoom
        zoom_range=0.05,
        # set range for random channel shifts
        channel_shift_range=0.07,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        rescale=1. / 255,
        # set rescaling factor (applied before any other transformation)
        # set function that will be applied on each input
        #Cutout Implementation
        preprocessing_function=get_random_eraser(p=0.08,v_l=0, v_h=0.3, s_l=0.008, s_h=0.05, pixel_level=False),
        #preprocessing_function=additional_augmenation, 
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

In [ ]:
img_width, img_height = 224,224
batch_size=32
test_datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=30,
        # randomly shift images horizontally
        width_shift_range=0.08,
        # randomly shift images vertically
        height_shift_range=0.05,
        # set range for random shear
        shear_range=0.07,
        # set range for random zoom
        zoom_range=0.05,
        # set range for random channel shifts
        channel_shift_range=0.07,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        rescale=1. / 255,
        # set rescaling factor (applied before any other transformation)
        # set function that will be applied on each input
        #Cutout Implementation
        preprocessing_function=get_random_eraser(p=0.08,v_l=0, v_h=0.3, s_l=0.008, s_h=0.05, pixel_level=False),
        #preprocessing_function=additional_augmenation, 
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

In [ ]:
#behaves weird, create a folder in content, move trainjpg there. Then create trainjpg here in content adn move from there
train_generator = train_datagen.flow_from_directory(
    directory=r"/content/trainjpg/",
    target_size=(224,224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 655 images belonging to 3 classes.


In [ ]:
#validation_datagen = ImageDataGenerator(rescale=1. / 255)


Found 220 images belonging to 3 classes.


In [ ]:
validation_generator = validation_datagen.flow_from_directory(
    directory=r"/content/validationjpg/",
    target_size=(224,224),
    color_mode="rgb",
    batch_size=32,
    shuffle=True,
    seed=42,
    class_mode='categorical')

Found 220 images belonging to 3 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory=r"/content/ttest/",
    target_size=(224,224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

Found 610 images belonging to 1 classes.


In [ ]:
def lr_schedule(epoch):

    lr = 1e-3
    if epoch > 90:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 60:
        lr *= 1e-2
    elif epoch > 40:
        lr *= 1e-1

    print('Learning Rate: ', lr)
    return lr

In [ ]:

from keras.callbacks import Callback
from keras import backend as K
import numpy as np

# Keras-Contib Implementation
class CyclicLR(Callback):

    def __init__(
            self,
            base_lr=0.001,
            max_lr=0.005,
            step_size=2000.,
            mode='triangular',
            gamma=1.,
            scale_fn=None,
            scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        if mode not in ['triangular', 'triangular2',
                        'exp_range']:
            raise KeyError("mode must be one of 'triangular', "
                           "'triangular2', or 'exp_range'")
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2.**(x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** x
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr is not None:
            self.base_lr = new_base_lr
        if new_max_lr is not None:
            self.max_lr = new_max_lr
        if new_step_size is not None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault(
            'lr', []).append(
            K.get_value(
                self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        print("Learning Rate: ", float(K.get_value(self.model.optimizer.lr)))
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [ ]:
class SaveWeights(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0 
        self.ep = 0  

    def on_epoch_end(self, epoch, logs={}):
        
        self.i += 1
        self.ep = epoch + 1
        #print("Self I: ", self.i, " Epoch: ", self.ep)
        if (self.i % 10) == 0:
              densenetmodel.save_weights(f"/content/gdrive/My Drive/densenet__{self.ep}.hdf5")
              print("Saved the Model after Epoch: ", self.i)
        
save_weight = SaveWeights()

In [ ]:


# Prepare callbacks for model saving and for learning rate adjustment.


filepath='/content/gdrive/My Drive/densenet:{epoch:03d}-validation.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath,
                             verbose=1,
                             save_best_only=True)

clr = CyclicLR(base_lr=0.001, 
               max_lr=0.006, 
               step_size=2000., 
               mode='triangular')

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, clr, save_weight]

In [ ]:
from keras import models
from keras import layers

In [ ]:
from keras.applications.densenet import DenseNet121
densenet_conv_base=DenseNet121(weights='imagenet',include_top=False,input_shape=(224,224,3))














29089792/29084464 [==============================] - 3s 0us/step


In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
model=densenet_conv_base.output

model=GlobalAveragePooling2D()(model)

predictions = Dense(3, activation= 'softmax')(model)
densenetmodel = Model(inputs = densenet_conv_base.input, outputs = predictions)

In [ ]:
densenet_conv_base.trainable=False

In [ ]:
opt = SGD(lr=0.001, momentum=0.9)
densenetmodel.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
nb_epoch=150
last_executed_epoch=0

print('Training for Epochs : ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " -------------------------------")

# # Fit the model on the batches generated by datagen.flow().
densenetmodel.fit_generator(generator=train_generator, validation_data=validation_generator, use_multiprocessing=True, epochs=nb_epoch, initial_epoch=last_executed_epoch, verbose=1, workers=6, callbacks=callbacks)

Training for Epochs : 1 - 150 -------------------------------
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/150
21/21 [==============================] - 110s 5s/step - loss: 1.0345 - acc: 0.4847 - val_loss: 0.6104 - val_acc: 0.7500

Epoch 00001: val_loss improved from inf to 0.61041, saving model to /content/gdrive/My Drive/densenet:001-validation.hdf5
Learning Rate:  0.001052499981597066
Epoch 2/150
21/21 [==============================] - 28s 1s/step - loss: 0.5063 - acc: 0.7927 - val_loss: 0.6415 - val_acc: 0.7182

Epoch 00002: val_loss did not improve from 0.61041
Learning Rate:  0.0011050000321120024
Epoch 3/150
20/21 [===========================>..] - ETA: 2s - loss: 0.3032 - acc: 0.9078
Epoch 00002: val_loss did not improve from 0.61041
21/21 [==============================] - 62s 3s/step - loss: 0.2971 - acc: 0.9106 - val_loss: 0.5888 - val_acc: 0.7455

Epoch 00003: val_loss improved from 0.61041 to 0.58879, saving model

Process ForkPoolWorker-573:
Process ForkPoolWorker-568:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-575:
Process ForkPoolWorker-572:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-565:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-576:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Process ForkPoolWorker-566:
Traceback (most recent call last):
Process ForkPoolWorker-574:
Process ForkPoolWorker-569:
Process ForkPoolWorker-570:
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/m

KeyboardInterrupt: ignored

In [ ]:
densenetmodel.load_weights('/content/gdrive/My Drive/densenet:004-validation.hdf5')
#Learning Rate:  0.0012100000167265534')

In [ ]:
scores=densenetmodel.evaluate_generator(validation_generator,verbose=1)
scores

7/7 [==============================] - 17s 2s/step


[0.6272263906218789, 0.8590909058397467]

In [ ]:
from keras.applications.vgg19 import VGG19
vgg19_conv_base=VGG19(weights='imagenet',include_top=False,input_shape=(224,224,3))





80142336/80134624 [==============================] - 7s 0us/step








In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
model=vgg19_conv_base.output

model=GlobalAveragePooling2D()(model)

predictions = Dense(3, activation= 'softmax')(model)
vgg19_freezemodel = Model(inputs = vgg19_conv_base.input, outputs = predictions)

In [ ]:
vgg19_conv_base.trainable=False

In [ ]:
opt = SGD(lr=0.001, momentum=0.9)
vgg19_freezemodel.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
nb_epoch=150
last_executed_epoch=0

print('Training for Epochs : ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " -------------------------------")

# # Fit the model on the batches generated by datagen.flow().
vgg19_freezemodel.fit_generator(generator=train_generator, validation_data=validation_generator, use_multiprocessing=True, epochs=nb_epoch, initial_epoch=last_executed_epoch, verbose=1, workers=6, callbacks=callbacks)

Training for Epochs : 1 - 150 -------------------------------
Epoch 1/150
21/21 [==============================] - 73s 3s/step - loss: 0.0582 - acc: 0.9806 - val_loss: 0.6097 - val_acc: 0.8364

Epoch 00001: val_loss improved from inf to 0.60972, saving model to /content/gdrive/My Drive/vgg19_32:001-validation.hdf5
Learning Rate:  0.001052499981597066
Epoch 2/150
21/21 [==============================] - 61s 3s/step - loss: 0.0260 - acc: 0.9940 - val_loss: 0.6825 - val_acc: 0.8136

Epoch 00002: val_loss did not improve from 0.60972
Learning Rate:  0.0011050000321120024
Epoch 3/150
21/21 [==============================] - 65s 3s/step - loss: 0.0176 - acc: 0.9955 - val_loss: 0.6386 - val_acc: 0.8273

Epoch 00003: val_loss did not improve from 0.60972
Learning Rate:  0.001157499966211617
Epoch 4/150
20/21 [===========================>..] - ETA: 3s - loss: 0.0222 - acc: 0.9938Learning Rate:  0.001157499966211617
Epoch 4/150
21/21 [==============================] - 66s 3s/step - loss: 0.0214 

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 16 could not be retrieved. It could be because a worker has died.
  UserWarning)


20/21 [===========================>..] - ETA: 3s - loss: 0.0075 - acc: 0.9953Learning Rate:  0.0018400000408291817
Epoch 17/150
21/21 [==============================] - 70s 3s/step - loss: 0.0084 - acc: 0.9955 - val_loss: 0.7778 - val_acc: 0.8318

Epoch 00017: val_loss did not improve from 0.60093
Learning Rate:  0.0018924999749287963
Epoch 18/150


Process ForkPoolWorker-1070:
Process ForkPoolWorker-1071:
Process ForkPoolWorker-1072:
Process ForkPoolWorker-1069:
Traceback (most recent call last):
Process ForkPoolWorker-1083:
Process ForkPoolWorker-1085:
Process ForkPoolWorker-1081:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-1073:
Process ForkPoolWorker-1082:
Process ForkPoolWorker-1084:
Process ForkPoolWorker-1074:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-1086:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Trac

Learning Rate:  0.0018924999749287963


  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
vgg19_freezemodel.load_weights("/content/gdrive/My Drive/vgg19_ffreeze-validation.hdf5")


In [ ]:
scores=vgg16_freezemodel.evaluate_generator(validation_generator,verbose=1)

14/14 [==============================] - 17s 1s/step


In [ ]:
#best scores after resnet 50 in zindi loss .40 is 0.50
#scores  this is vgg16 score

[0.3809942907907746, 0.8681818181818182]

In [ ]:
#find optimal learning rate and fix this learning rate for further training


In [ ]:
test_generator.reset()
a_predictions=vgg16_freezemodel.predict_generator(test_generator,verbose=1)

39/39 [==============================] - 48s 1s/step


In [ ]:
a_data=a_predictions.round(decimals=3)
a_data[0:6]

array([[0.   , 0.992, 0.008],
       [0.   , 0.491, 0.509],
       [0.   , 0.987, 0.013],
       [0.955, 0.035, 0.01 ],
       [0.993, 0.005, 0.002],
       [0.   , 0.   , 1.   ]], dtype=float32)

In [ ]:
predicted_class_indices=np.argmax(a_predictions,axis=1)
labels = (train_generator.class_indices)
labels1 = dict((v,k) for k,v in labels.items())
pred = [labels1[k] for k in predicted_class_indices]

In [ ]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":pred})

In [ ]:
def get_ids(filenames):
  for i in filenames:
    k=i.split('/')
    kk=k[1].split('.')
    return kk[0]

In [ ]:
results['Filename']=results.apply(lambda x: get_ids(x),axis=1)

In [ ]:
leaf_rust=pd.Series(range(610),name='leaf_rust',dtype=np.float32)
stem_rust=pd.Series(range(610),name='stem_rust',dtype=np.float32)
healthy_wheat=pd.Series(range(610),name='healthy_wheat',dtype=np.float32)
sub=pd.concat([healthy_wheat,leaf_rust,stem_rust],axis=1)
sub.shape

(610, 3)

In [ ]:
for i in tqdm(range(0,len(data))):
  sub.loc[i]=data[i]

100%|██████████| 610/610 [00:00<00:00, 5036.73it/s]


In [ ]:
sub["ID"]=results['Filename']

In [ ]:
submit=sub[['ID','leaf_rust','stem_rust','healthy_wheat']]

In [ ]:
submit.head(3)

,ID,leaf_rust,stem_rust,healthy_wheat
0,008FWT,0.965,0.035,0.000
1,00AQXY,0.153,0.846,0.001
2,01OJZX,0.966,0.034,0.000


In [ ]:
submit.to_csv('vgg161.csv',index=False)